In [ ]:
from glob import glob
from os.path import join
import os
import numpy as np
import itertools
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
import matplotlib.pyplot as plt
import pandas as pd

# Preprocessed data reading

In [ ]:
data_folder = join('..', 'data', 'preprocessed_data')

In [ ]:
data_files = glob(join(data_folder, '*.csv'))

In [ ]:
timeseries = []

for f in data_files:
    timeseries.append(pd.read_csv(f, index_col=0))

# ML pipeline

In [ ]:
pca = PCA()

In [ ]:
kernel = DotProduct() + WhiteKernel()

In [ ]:
gpr = GaussianProcessRegressor(kernel=kernel, random_state=0)

In [ ]:
pipe = Pipeline(steps=[('pca', pca), ('gpr', gpr)])

In [ ]:
param_grid = {
    'pca__n_components': [1, 2, 3]
}

In [ ]:
search = GridSearchCV(pipe, param_grid, n_jobs=-1)

# Test single timeserie

## Data extraction

In [ ]:
x_header = ['ExactoKnife_' + axis for axis in ['x', 'y', 'z']]

In [ ]:
y_header = ['opt_force_' + axis for axis in ['x', 'y', 'z']]

In [ ]:
X = timeseries[0][x_header]

In [ ]:
y1 = timeseries[0][y_header] # multiple dimensions force

In [ ]:
y2 = np.linalg.norm(y1.values,axis=1) # single dimension normed force

## Best parameter search

In [ ]:
search.fit(X, y1)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

In [ ]:
search.fit(X, y2)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)